# Creación de Splits (Patient-Level - SIN DATA LEAKAGE)

**Split de 3 vías:** Train (60%) / Dev (20%) / Test (20%)

**Problema anterior:**
- Split por casos → Mismo paciente en train Y val
- 90 pacientes con 35 consultas promedio c/u
- 100% leakage (modelo memoriza pacientes)

**Solución:**
- Split por Prontuario (patient ID)
- 54 pacientes train (60%) / 18 pacientes dev (20%) / 18 pacientes test (20%)
- 0% leakage (pacientes disjuntos entre los 3 conjuntos)

**Estratificación:**
- Estratificar por CLASE MAYORITARIA del paciente
- Si paciente tiene 10 consultas ansiedad + 2 depresión → etiqueta = ansiedad
- Garantiza distribución similar en train/dev/test

**Justificación del split de 3 vías:**
- **Train (60%)**: Análisis exploratorio y desarrollo de Concept_PY
- **Dev (20%)**: Iteración y refinamiento (puede mirarse múltiples veces)
- **Test (20%)**: Evaluación final CIEGA (solo una vez al final)
- Evita overfitting indirecto al desarrollar reglas basadas en conocimiento

In [10]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from collections import Counter

# Configuración
DATA_PATH = Path('../data')
SPLITS_PATH = DATA_PATH / 'splits'
SPLITS_PATH.mkdir(exist_ok=True)

RANDOM_STATE = 42
DEV_SIZE = 0.2   # 20% para dev
TEST_SIZE = 0.2  # 20% para test
# Train será el restante: 1 - 0.2 - 0.2 = 60%

# Columnas de ips_clean.csv
PATIENT_COL = 'id_paciente'
LABEL_COL = 'etiqueta'
TEXT_COL = 'texto'

print(f"Random state: {RANDOM_STATE}")
print(f"Split ratio: Train 60% / Dev 20% / Test 20%")

Random state: 42
Split ratio: Train 60% / Dev 20% / Test 20%


## 1. Cargar dataset

**IMPORTANTE:** Usar `ips_clean.csv` (3,127 casos) generado por `01_eda_understanding.ipynb`
- Dataset limpio con remoción de 43,938 oraciones duplicadas
- Garantiza comparación justa con baselines anteriores (80/20)

In [11]:
# Cargar ips_clean.csv (dataset limpio sin duplicados)
INPUT_FILE = DATA_PATH / 'ips_clean.csv'
df = pd.read_csv(INPUT_FILE)

print(f"Dataset cargado: {INPUT_FILE}")
print(f"Total casos: {len(df)}")
print(f"Columnas: {list(df.columns)}")
print(f"\nDistribución clases:")
print(df[LABEL_COL].value_counts())

Dataset cargado: ../data/ips_clean.csv
Total casos: 3127
Columnas: ['id_paciente', 'fecha', 'etiqueta', 'texto']

Distribución clases:
etiqueta
depresion    2202
ansiedad      925
Name: count, dtype: int64


## 2. Análisis a nivel paciente

In [12]:
# Estadísticas por paciente
n_patients = df[PATIENT_COL].nunique()
n_cases = len(df)
avg_cases = n_cases / n_patients

patient_counts = df[PATIENT_COL].value_counts()

print("=" * 60)
print("ANÁLISIS A NIVEL PACIENTE")
print("=" * 60)
print(f"Pacientes únicos: {n_patients}")
print(f"Casos totales: {n_cases}")
print(f"Promedio casos/paciente: {avg_cases:.2f}")
print(f"Mediana casos/paciente: {patient_counts.median():.0f}")
print(f"Min-Max casos/paciente: {patient_counts.min()}-{patient_counts.max()}")

# Distribución
print(f"\nDistribución casos/paciente:")
print(patient_counts.describe())

ANÁLISIS A NIVEL PACIENTE
Pacientes únicos: 90
Casos totales: 3127
Promedio casos/paciente: 34.74
Mediana casos/paciente: 33
Min-Max casos/paciente: 6-63

Distribución casos/paciente:
count    90.000000
mean     34.744444
std      11.404278
min       6.000000
25%      27.000000
50%      33.000000
75%      42.000000
max      63.000000
Name: count, dtype: float64


## 3. Asignar clase mayoritaria por paciente

**Estrategia de estratificación:**
- Cada paciente tiene etiqueta = clase con más consultas
- Ejemplo: Paciente A con 8 consultas ansiedad + 2 depresión → etiqueta_paciente = 'ansiedad'
- Esto permite estratificar el split de pacientes

In [13]:
def get_patient_majority_label(patient_id, df, label_col):
    """Obtiene la clase mayoritaria de un paciente"""
    patient_labels = df[df[PATIENT_COL] == patient_id][label_col]
    most_common = Counter(patient_labels).most_common(1)[0][0]
    return most_common

# Crear DataFrame de pacientes con su clase mayoritaria
patients = df[PATIENT_COL].unique()
patient_labels = {p: get_patient_majority_label(p, df, LABEL_COL) for p in patients}

patients_df = pd.DataFrame({
    'Prontuario': list(patient_labels.keys()),
    'etiqueta_mayoritaria': list(patient_labels.values())
})

print("Distribución de pacientes por clase mayoritaria:")
print(patients_df['etiqueta_mayoritaria'].value_counts())
print(f"\nRatio ansiedad/depresión (pacientes): {patients_df['etiqueta_mayoritaria'].value_counts()['ansiedad'] / patients_df['etiqueta_mayoritaria'].value_counts()['depresion']:.2f}")

Distribución de pacientes por clase mayoritaria:
etiqueta_mayoritaria
depresion    57
ansiedad     33
Name: count, dtype: int64

Ratio ansiedad/depresión (pacientes): 0.58


## 4. Split por PACIENTES en 3 conjuntos (estratificado)

**CRÍTICO:** Dividir pacientes (no casos) para evitar data leakage

**Estrategia:**
1. Primero: separar test (20%) del total
2. Segundo: del restante (80%), separar train (75% de 80% = 60% total) y dev (25% de 80% = 20% total)
3. Resultado: Train 60% / Dev 20% / Test 20%

In [14]:
# Split de pacientes en 3 conjuntos (60/20/20)

# Paso 1: Separar test set (20%) del total
train_dev_patients, test_patients = train_test_split(
    patients_df['Prontuario'],
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
    stratify=patients_df['etiqueta_mayoritaria']
)

# Paso 2: Del restante (80%), separar train (75%) y dev (25%)
# 75% de 80% = 60% del total
# 25% de 80% = 20% del total
train_patients, dev_patients = train_test_split(
    train_dev_patients,
    test_size=0.25,  # 25% del 80% = 20% del total
    random_state=RANDOM_STATE,
    stratify=patients_df[patients_df['Prontuario'].isin(train_dev_patients)]['etiqueta_mayoritaria']
)

print("=" * 60)
print("SPLIT POR PACIENTES (PATIENT-LEVEL) - 3 CONJUNTOS")
print("=" * 60)
print(f"Pacientes en train: {len(train_patients)} ({len(train_patients)/n_patients*100:.1f}%)")
print(f"Pacientes en dev:   {len(dev_patients)} ({len(dev_patients)/n_patients*100:.1f}%)")
print(f"Pacientes en test:  {len(test_patients)} ({len(test_patients)/n_patients*100:.1f}%)")

# Verificar no overlap
overlap_train_dev = set(train_patients) & set(dev_patients)
overlap_train_test = set(train_patients) & set(test_patients)
overlap_dev_test = set(dev_patients) & set(test_patients)

print(f"\n✓ Overlap train-dev: {len(overlap_train_dev)} pacientes (DEBE SER 0)")
print(f"✓ Overlap train-test: {len(overlap_train_test)} pacientes (DEBE SER 0)")
print(f"✓ Overlap dev-test: {len(overlap_dev_test)} pacientes (DEBE SER 0)")

assert len(overlap_train_dev) == 0, "ERROR: Hay pacientes en train Y dev"
assert len(overlap_train_test) == 0, "ERROR: Hay pacientes en train Y test"
assert len(overlap_dev_test) == 0, "ERROR: Hay pacientes en dev Y test"

SPLIT POR PACIENTES (PATIENT-LEVEL) - 3 CONJUNTOS
Pacientes en train: 54 (60.0%)
Pacientes en dev:   18 (20.0%)
Pacientes en test:  18 (20.0%)

✓ Overlap train-dev: 0 pacientes (DEBE SER 0)
✓ Overlap train-test: 0 pacientes (DEBE SER 0)
✓ Overlap dev-test: 0 pacientes (DEBE SER 0)


## 5. Obtener índices de casos (train/dev/test)

Ahora convertimos pacientes → casos (consultas)

In [15]:
# Filtrar casos por pacientes asignados
train_mask = df[PATIENT_COL].isin(train_patients)
dev_mask = df[PATIENT_COL].isin(dev_patients)
test_mask = df[PATIENT_COL].isin(test_patients)

train_idx = df[train_mask].index.tolist()
dev_idx = df[dev_mask].index.tolist()
test_idx = df[test_mask].index.tolist()

print("=" * 60)
print("CASOS RESULTANTES")
print("=" * 60)
print(f"Casos en train: {len(train_idx)} ({len(train_idx)/n_cases*100:.1f}%)")
print(f"Casos en dev:   {len(dev_idx)} ({len(dev_idx)/n_cases*100:.1f}%)")
print(f"Casos en test:  {len(test_idx)} ({len(test_idx)/n_cases*100:.1f}%)")

# Distribución de clases en train/dev/test
train_labels = df.loc[train_idx, LABEL_COL]
dev_labels = df.loc[dev_idx, LABEL_COL]
test_labels = df.loc[test_idx, LABEL_COL]

print(f"\nDistribución train:")
print(train_labels.value_counts())
print(f"Ratio A/D: {train_labels.value_counts()['ansiedad'] / train_labels.value_counts()['depresion']:.2f}")

print(f"\nDistribución dev:")
print(dev_labels.value_counts())
print(f"Ratio A/D: {dev_labels.value_counts()['ansiedad'] / dev_labels.value_counts()['depresion']:.2f}")

print(f"\nDistribución test:")
print(test_labels.value_counts())
print(f"Ratio A/D: {test_labels.value_counts()['ansiedad'] / test_labels.value_counts()['depresion']:.2f}")

CASOS RESULTANTES
Casos en train: 1849 (59.1%)
Casos en dev:   641 (20.5%)
Casos en test:  637 (20.4%)

Distribución train:
etiqueta
depresion    1270
ansiedad      579
Name: count, dtype: int64
Ratio A/D: 0.46

Distribución dev:
etiqueta
depresion    456
ansiedad     185
Name: count, dtype: int64
Ratio A/D: 0.41

Distribución test:
etiqueta
depresion    476
ansiedad     161
Name: count, dtype: int64
Ratio A/D: 0.34


## 6. Verificación: No data leakage

In [16]:
# Verificar que no hay pacientes compartidos
train_patients_check = set(df.loc[train_idx, PATIENT_COL])
dev_patients_check = set(df.loc[dev_idx, PATIENT_COL])
test_patients_check = set(df.loc[test_idx, PATIENT_COL])

overlap_train_dev = train_patients_check & dev_patients_check
overlap_train_test = train_patients_check & test_patients_check
overlap_dev_test = dev_patients_check & test_patients_check

print("=" * 60)
print("VERIFICACIÓN DATA LEAKAGE")
print("=" * 60)
print(f"Pacientes únicos en train: {len(train_patients_check)}")
print(f"Pacientes únicos en dev:   {len(dev_patients_check)}")
print(f"Pacientes únicos en test:  {len(test_patients_check)}")
print(f"\nPacientes compartidos:")
print(f"  - train ∩ dev:  {len(overlap_train_dev)}")
print(f"  - train ∩ test: {len(overlap_train_test)}")
print(f"  - dev ∩ test:   {len(overlap_dev_test)}")

if len(overlap_train_dev) == 0 and len(overlap_train_test) == 0 and len(overlap_dev_test) == 0:
    print("\n✅ VERIFICACIÓN EXITOSA: NO HAY DATA LEAKAGE")
else:
    print(f"\n❌ ERROR: Hay pacientes compartidos entre conjuntos")
    if len(overlap_train_dev) > 0:
        print(f"Train-Dev overlap: {list(overlap_train_dev)[:5]}")
    if len(overlap_train_test) > 0:
        print(f"Train-Test overlap: {list(overlap_train_test)[:5]}")
    if len(overlap_dev_test) > 0:
        print(f"Dev-Test overlap: {list(overlap_dev_test)[:5]}")
    raise ValueError("DATA LEAKAGE DETECTADO")

VERIFICACIÓN DATA LEAKAGE
Pacientes únicos en train: 54
Pacientes únicos en dev:   18
Pacientes únicos en test:  18

Pacientes compartidos:
  - train ∩ dev:  0
  - train ∩ test: 0
  - dev ∩ test:   0

✅ VERIFICACIÓN EXITOSA: NO HAY DATA LEAKAGE


## 7. Guardar splits

**IMPORTANTE:** Crear `row_id` único para cada caso (necesario para merge en baselines)

In [17]:
# Asignar row_id como índice original
df['row_id'] = df.index

# Preparar dataset base
df_base = df[['row_id', PATIENT_COL, TEXT_COL, LABEL_COL]].copy()
df_base.columns = ['row_id', 'patient_id', 'texto', 'etiqueta']

# Normalizar etiquetas
label_map = {'depresivo': 'depresion'}  # Mapeo si hay variantes
df_base['etiqueta'] = df_base['etiqueta'].str.lower().map(lambda x: label_map.get(x, x))

# Guardar archivos
dataset_base_path = SPLITS_PATH / 'dataset_base.csv'
train_indices_path = SPLITS_PATH / 'train_indices.csv'
dev_indices_path = SPLITS_PATH / 'dev_indices.csv'
test_indices_path = SPLITS_PATH / 'test_indices.csv'

df_base.to_csv(dataset_base_path, index=False, encoding='utf-8')
pd.DataFrame({'row_id': train_idx}).to_csv(train_indices_path, index=False)
pd.DataFrame({'row_id': dev_idx}).to_csv(dev_indices_path, index=False)
pd.DataFrame({'row_id': test_idx}).to_csv(test_indices_path, index=False)

print("=" * 60)
print("ARCHIVOS GUARDADOS")
print("=" * 60)
print(f"Ubicación: {SPLITS_PATH}/")
print(f"\n1. dataset_base.csv")
print(f"   - {len(df_base)} casos")
print(f"   - Columnas: row_id, patient_id, texto, etiqueta")
print(f"\n2. train_indices.csv")
print(f"   - {len(train_idx)} índices (de {len(train_patients)} pacientes)")
print(f"\n3. dev_indices.csv")
print(f"   - {len(dev_idx)} índices (de {len(dev_patients)} pacientes)")
print(f"\n4. test_indices.csv")
print(f"   - {len(test_idx)} índices (de {len(test_patients)} pacientes)")

ARCHIVOS GUARDADOS
Ubicación: ../data/splits/

1. dataset_base.csv
   - 3127 casos
   - Columnas: row_id, patient_id, texto, etiqueta

2. train_indices.csv
   - 1849 índices (de 54 pacientes)

3. dev_indices.csv
   - 641 índices (de 18 pacientes)

4. test_indices.csv
   - 637 índices (de 18 pacientes)


## 8. Reporte final y comparación

Comparar distribución antes/después de split

In [18]:
print("\n" + "=" * 60)
print("RESUMEN: SPLIT POR PACIENTES (SIN LEAKAGE) - 3 CONJUNTOS")
print("=" * 60)

print(f"\nDATASET COMPLETO:")
print(f"  - {n_patients} pacientes")
print(f"  - {n_cases} casos (consultas)")
print(f"  - {avg_cases:.1f} casos/paciente promedio")

print(f"\nSPLIT REALIZADO (60/20/20):")
print(f"  - Train: {len(train_patients)} pacientes ({len(train_idx)} casos) - {len(train_idx)/n_cases*100:.1f}%")
print(f"  - Dev:   {len(dev_patients)} pacientes ({len(dev_idx)} casos) - {len(dev_idx)/n_cases*100:.1f}%")
print(f"  - Test:  {len(test_patients)} pacientes ({len(test_idx)} casos) - {len(test_idx)/n_cases*100:.1f}%")

print(f"\nPROPÓSITO DE CADA CONJUNTO:")
print(f"  - Train: Análisis exploratorio, desarrollo de Concept_PY")
print(f"  - Dev:   Validación iterativa, ajuste de reglas (puede verse múltiples veces)")
print(f"  - Test:  Evaluación final CIEGA (solo una vez, comparación con baselines)")

print(f"\nVERIFICACIÓN:")
print(f"  - Pacientes compartidos train-dev: {len(overlap_train_dev)} (0 = correcto)")
print(f"  - Pacientes compartidos train-test: {len(overlap_train_test)} (0 = correcto)")
print(f"  - Pacientes compartidos dev-test: {len(overlap_dev_test)} (0 = correcto)")
print(f"  - Estratificación por clase mayoritaria: ✓")
print(f"  - Random seed fijo: {RANDOM_STATE} (reproducible)")

print(f"\nCAMBIO vs SPLIT ANTERIOR (80/20):")
print(f"  - Antes: 2 conjuntos (train 80% / val 20%)")
print(f"  - Ahora: 3 conjuntos (train 60% / dev 20% / test 20%)")
print(f"  - Ventaja: Evita overfitting al desarrollar Concept_PY")
print(f"  - Dev set permite iterar sin contaminar test")

print(f"\nARCHIVOS PARA BASELINES:")
print(f"  1. Cargar {dataset_base_path.name}")
print(f"  2. Cargar {train_indices_path.name} y {dev_indices_path.name}")
print(f"  3. Filtrar usando row_id")
print(f"  4. Entrenar con train, evaluar con dev")
print(f"  5. Test se reserva para comparación final")

print("\n" + "=" * 60)
print("IMPORTANTE: RE-EJECUTAR TODOS LOS BASELINES con nuevos splits")
print("para actualizar métricas y mantener comparabilidad")
print("=" * 60)


RESUMEN: SPLIT POR PACIENTES (SIN LEAKAGE) - 3 CONJUNTOS

DATASET COMPLETO:
  - 90 pacientes
  - 3127 casos (consultas)
  - 34.7 casos/paciente promedio

SPLIT REALIZADO (60/20/20):
  - Train: 54 pacientes (1849 casos) - 59.1%
  - Dev:   18 pacientes (641 casos) - 20.5%
  - Test:  18 pacientes (637 casos) - 20.4%

PROPÓSITO DE CADA CONJUNTO:
  - Train: Análisis exploratorio, desarrollo de Concept_PY
  - Dev:   Validación iterativa, ajuste de reglas (puede verse múltiples veces)
  - Test:  Evaluación final CIEGA (solo una vez, comparación con baselines)

VERIFICACIÓN:
  - Pacientes compartidos train-dev: 0 (0 = correcto)
  - Pacientes compartidos train-test: 0 (0 = correcto)
  - Pacientes compartidos dev-test: 0 (0 = correcto)
  - Estratificación por clase mayoritaria: ✓
  - Random seed fijo: 42 (reproducible)

CAMBIO vs SPLIT ANTERIOR (80/20):
  - Antes: 2 conjuntos (train 80% / val 20%)
  - Ahora: 3 conjuntos (train 60% / dev 20% / test 20%)
  - Ventaja: Evita overfitting al desarrol